In [14]:
from resources.filterLang import FilterLanguage as FL
from resources.tokTT import CommentTokenizer as CT 
import fasttext
import numpy as np
from resources.basicIO import InputOutput as IO

In [15]:
def NN(model, line, K):
    return model.get_nearest_neighbors(line, k=K)

In [16]:
# finds similarity score 
def score(model, line,k=10):
    # words contains all the words in the corpus
    lst1 = NN(model, line, k)
    v1 = []
    l1 = [x[1] for x in lst1]
    l10 = [x[0] for x in lst1]
    for i in range(len(model.words)):
        try:
            v1.append(l10[l1.index(model.words[i])])
        except:
            v1.append(0)
    return v1


In [17]:
def cos_sim(a, b):
	dot_product = np.dot(a, b)
	norm_a = np.linalg.norm(a)
	norm_b = np.linalg.norm(b)
	return dot_product / (norm_a * norm_b)

In [18]:
def sim(x,y,sim_type='cosine_sim'):

    if(sim_type == 'cosine_sim'):
        return cos_sim(x,y)

In [19]:
# find similarity score matrix between A and B
# pass transpose of B
def sim_matrix(A,B,sim_type):
    m,p = A.shape
    p,n = B.shape
    C = np.zeros((m,n))
    for i in range(m):
        for j in range(n):
            C[i][j] = sim(A[i,:], B[:,j],sim_type)
    return C

In [20]:
# finds classwise average and returns an array of predicted
# class labels. labels are classwise labels in ascending order
def label_avg(X,labels):

    m,n = X.shape
    if(m != len(labels)):
        return None

    avg_sums = [] 
    no_labels = len(list(set(labels)))

    for p in range(n):

        column = X[:, p]

        avgs = np.zeros(no_labels)

        for lbl in range(no_labels):
            indices = [ix for ix, label in enumerate(labels) if label == lbl]
            avgs[lbl] = np.mean([column[x] for x in indices])
    
        max_avg_pos = avgs.argmax()
        avg_sums.append(max_avg_pos)

    return avg_sums


In [21]:
model_N_2 = fasttext.load_model('models/ft_unsupervised_N_2.bin')

In [22]:
# loading 60 comments
expansion_text = IO.load_csv_col('datasets/random_sample.csv','comment')
expansion_text = expansion_text[0:60]
expansion_text_labels = IO.load_csv_col('datasets/random_sample.csv','label')
expansion_text_labels = list(map(str,map(int,expansion_text_labels[0:60])))
seed_set_text = IO.load_text('datasets/seed_set.txt')
seed_set_labels = IO.load_text('datasets/seed_set_labels.txt')
seed_set_labels = list(map(int, seed_set_labels[0:60]))
seed_set_TK = CT.cleaned('datasets/seed_set.txt')
expansion_TK = [CT.tokenize(x) for x in expansion_text]


In [23]:
score_seed = []
i = 0
for line in seed_set_TK:
    score_seed.append(score(model_N_2, line, 20))
    i += 1
    if(i == 60):
        break


In [24]:
score_ex = []
i=0
for line in expansion_TK:
    score_ex.append(score(model_N_2,line,50))
    i+=1
    if(i==60):
        break

In [25]:
A = np.array(score_seed)
B = np.array(score_ex).T
C = sim_matrix(A,B,'cosine_sim')

In [26]:
Y = label_avg(C,seed_set_labels)
print(Y)

[1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1]
